In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [45]:
folder = "../../Data/hpc_out/rwm_wip_2_6r7m/"
mol_type = "6r7m"
#folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
                println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                continue
            end
            
            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end

E_diff = 9.873233621632608
E_diff = 5.472242272278891
E_diff = 7.146086029048092
E_diff = 8.55306337546972
JLD2.InvalidDataException("Did not find a Superblock.")
Error in 70_rwm_wip_2_6r7m.jld2
E_diff = 0.22657185415118875
JLD2.InvalidDataException("Did not find a Superblock.")
Error in 8_rwm_wip_2_6r7m.jld2
JLD2.InvalidDataException("Did not find a Superblock.")
Error in 91_rwm_wip_2_6r7m.jld2


In [49]:
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")

Minimal configuration id: 79 | Average acceptance rate: 0.2874883173000334


In [50]:
for elem in evaluation_strings
    println(elem)
end


2: 1.5 | 0.13123639 | 8219.631762155439, 8182.068548969937, 8186.883490596136 | 12.822131954071697


5: 1.5 | 0.10519283 | 8219.581581761831, 8181.94108587442, 8181.94108587442 | 19.53017961077969
6: 1.5 | 0.029509477 | 8219.724793443385, 8190.19479730255, 8194.126557866484 | 18.867774737756072
7: 2.3 | 0.4617816 | 8219.631018412894, 8190.564741493522, 8196.504949585807 | 9.460925060734464

9: 2.3 | 0.5759727 | 8219.581903423175, 8180.5464200927345, 8190.163580010795 | 12.35281827345011

11: 1.5 | 0.0110147325 | 8219.651553571002, 8175.236303699988, 8175.236303699988 | 14.49041782851192
12: 1.5 | 0.50873584 | 8219.651213030318, 8191.3752629361325, 8202.047902905517 | 33.068640612002504
13: 2.3 | 0.78055316 | 9193.538887652396, 8190.653042226419, 8200.111942127534 | 6.143810524715297

15: 1.5 | 0.013515138 | 8205.56510900288, 8185.128417735978, 8203.819485801294 | 26.494643083542936

17: 1.5 | 0.602004 | 8219.638007019686, 8195.446467201846, 8199.6067422482 | 24.176226444022667
18: 1.5

In [48]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

JLD2.InvalidDataException("Did not find a Superblock.")
JLD2.InvalidDataException("Did not find a Superblock.")
JLD2.InvalidDataException("Did not find a Superblock.")


# Print Single Assembly Path

In [54]:
@load "../../Data/hpc_out/rwm_wip_2_6r7m/79_rwm_wip_2_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [55]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
